# Table of Contents
 <p><div class="lev1 toc-item"><a href="#Cеминар-3.-Методы-обработки-данных" data-toc-modified-id="Cеминар-3.-Методы-обработки-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Cеминар 3. Методы обработки данных</a></div><div class="lev1 toc-item"><a href="#Загрузка-данных-и-&quot;выбор&quot;-задачи" data-toc-modified-id="Загрузка-данных-и-&quot;выбор&quot;-задачи-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Загрузка данных и "выбор" задачи</a></div><div class="lev2 toc-item"><a href="#Предобработка" data-toc-modified-id="Предобработка-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Предобработка</a></div><div class="lev3 toc-item"><a href="#Новая-категориальная-переменная" data-toc-modified-id="Новая-категориальная-переменная-2.1.1"><span class="toc-item-num">2.1.1&nbsp;&nbsp;</span>Новая категориальная переменная</a></div><div class="lev3 toc-item"><a href="#Label-encoder" data-toc-modified-id="Label-encoder-2.1.2"><span class="toc-item-num">2.1.2&nbsp;&nbsp;</span>Label encoder</a></div><div class="lev3 toc-item"><a href="#One-hot-encoding" data-toc-modified-id="One-hot-encoding-2.1.3"><span class="toc-item-num">2.1.3&nbsp;&nbsp;</span>One-hot-encoding</a></div><div class="lev3 toc-item"><a href="#Более-умная-кодировка" data-toc-modified-id="Более-умная-кодировка-2.1.4"><span class="toc-item-num">2.1.4&nbsp;&nbsp;</span>Более умная кодировка</a></div><div class="lev4 toc-item"><a href="#Среднее-по-категории" data-toc-modified-id="Среднее-по-категории-2.1.4.1"><span class="toc-item-num">2.1.4.1&nbsp;&nbsp;</span>Среднее по категории</a></div><div class="lev4 toc-item"><a href="#Вероятность-категории" data-toc-modified-id="Вероятность-категории-2.1.4.2"><span class="toc-item-num">2.1.4.2&nbsp;&nbsp;</span>Вероятность категории</a></div><div class="lev4 toc-item"><a href="#TF-IDF-для-категориальных-переменных" data-toc-modified-id="TF-IDF-для-категориальных-переменных-2.1.4.3"><span class="toc-item-num">2.1.4.3&nbsp;&nbsp;</span>TF-IDF для категориальных переменных</a></div><div class="lev3 toc-item"><a href="#Разложение-количественной-переменной-в-&quot;корзины&quot;" data-toc-modified-id="Разложение-количественной-переменной-в-&quot;корзины&quot;-2.1.5"><span class="toc-item-num">2.1.5&nbsp;&nbsp;</span>Разложение количественной переменной в "корзины"</a></div><div class="lev1 toc-item"><a href="#Собственно,-кластеризация" data-toc-modified-id="Собственно,-кластеризация-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Собственно, кластеризация</a></div>

# Cеминар 3. Методы обработки данных

Сегодняшний семинар будет более практическим. На примере одного набора данных мы постараемся охватить как можно больше методов предобработки данных.

Преждем чем приступать к обработке, надо определиться с тем, какую именно задачу мы будем решать.

In [67]:
import pandas as pd
import scipy
import numpy as np
import matplotlib.pyplot as plt

plt.style.use('ggplot')

%matplotlib inline

plt.rcParams['figure.figsize'] = (10, 6)

# Загрузка данных и "выбор" задачи

In [68]:
df_offers = pd.read_excel("./WineKMC.xlsx", sheetname=0)
df_offers.columns = ["offer_id", "campaign", "varietal", "min_qty", "discount", "origin", "past_peak"]
df_offers.head()

,offer_id,campaign,varietal,min_qty,discount,origin,past_peak
0,1,January,Malbec,72,56,France,False
1,2,January,Pinot Noir,72,17,France,False
2,3,February,Espumante,144,32,Oregon,True
3,4,February,Champagne,72,48,France,True
4,5,February,Cabernet Sauvignon,144,44,New Zealand,True


In [69]:
df_transactions = pd.read_excel("./WineKMC.xlsx", sheetname=1)
df_transactions.columns = ["customer_name", "offer_id"]
df_transactions.head()

,customer_name,offer_id
0,Smith,2
1,Smith,24
2,Johnson,17
3,Johnson,24
4,Johnson,26


Данные состоят из двух таблиц - транзакции по промо акциям и данные о самих промо-акциях. В исходных данных нет никакой разметки, и придумывать её тоже не хочется. Давайте попробуем решить задачу кластеризации

In [70]:
df = df_transactions.merge(df_offers, on='offer_id')

In [71]:
df_start = df
df.head()

,customer_name,offer_id,campaign,varietal,min_qty,discount,origin,past_peak
0,Smith,2,January,Pinot Noir,72,17,France,False
1,Rodriguez,2,January,Pinot Noir,72,17,France,False
2,Martin,2,January,Pinot Noir,72,17,France,False
3,Jackson,2,January,Pinot Noir,72,17,France,False
4,Campbell,2,January,Pinot Noir,72,17,France,False


## Предобработка

### Новая категориальная переменная

In [12]:
df.loc[:, 'varietal + origin'] = df.loc[:, 'varietal'] + '+' + df.loc[:, 'origin']

In [13]:
df.head()

,customer_name,offer_id,campaign,varietal,min_qty,discount,origin,past_peak,varietal + origin
0,Smith,2,January,Pinot Noir,72,17,France,False,Pinot Noir+France
1,Rodriguez,2,January,Pinot Noir,72,17,France,False,Pinot Noir+France
2,Martin,2,January,Pinot Noir,72,17,France,False,Pinot Noir+France
3,Jackson,2,January,Pinot Noir,72,17,France,False,Pinot Noir+France
4,Campbell,2,January,Pinot Noir,72,17,France,False,Pinot Noir+France


In [ ]:
df.loc[:, varietal]

In [14]:
# Для универсальности можно реализовать функцию
def make_conj_feature(df, feature1, feature2):
    new_feature_name = feature1 + '_' + feature2
    df.loc[:, new_feature_name] = df.loc[:, feature1] + '+' + df.loc[:, feature2]
    return df

df = make_conj_feature(df, 'varietal', 'origin')

# Но лучше так
df = df.pipe(make_conj_feature, 'varietal', 'origin')

In [15]:
df.head()

,customer_name,offer_id,campaign,varietal,min_qty,discount,origin,past_peak,varietal + origin,varietal_origin
0,Smith,2,January,Pinot Noir,72,17,France,False,Pinot Noir+France,Pinot Noir+France
1,Rodriguez,2,January,Pinot Noir,72,17,France,False,Pinot Noir+France,Pinot Noir+France
2,Martin,2,January,Pinot Noir,72,17,France,False,Pinot Noir+France,Pinot Noir+France
3,Jackson,2,January,Pinot Noir,72,17,France,False,Pinot Noir+France,Pinot Noir+France
4,Campbell,2,January,Pinot Noir,72,17,France,False,Pinot Noir+France,Pinot Noir+France


### Label encoder

Самый бездарный кодировщик.

Пожалуй имеет смысл, только если вы заранее можете задать категории интерпретируемое числовое значение.

In [19]:
from sklearn.preprocessing import LabelEncoder

lab_enc = LabelEncoder()
temp = lab_enc.fit_transform(df.origin)
df.loc[:, 'origin_le'] = temp

In [22]:
lab_enc.classes_

array(['Australia', 'California', 'Chile', 'France', 'Germany', 'Italy',
       'New Zealand', 'Oregon', 'South Africa'], dtype=object)

In [25]:
df.head()
# кодировщик не очень полезен, потому что нумерация классов происходит случайно, просто вместо текста мы получаем цифры
# (но мы также не можем применять регрессию к ним, очевидно)

,customer_name,offer_id,campaign,varietal,min_qty,discount,origin,past_peak,varietal + origin,varietal_origin,origin_le
0,Smith,2,January,Pinot Noir,72,17,France,False,Pinot Noir+France,Pinot Noir+France,3
1,Rodriguez,2,January,Pinot Noir,72,17,France,False,Pinot Noir+France,Pinot Noir+France,3
2,Martin,2,January,Pinot Noir,72,17,France,False,Pinot Noir+France,Pinot Noir+France,3
3,Jackson,2,January,Pinot Noir,72,17,France,False,Pinot Noir+France,Pinot Noir+France,3
4,Campbell,2,January,Pinot Noir,72,17,France,False,Pinot Noir+France,Pinot Noir+France,3


### One-hot-encoding

In [26]:
from sklearn.preprocessing import OneHotEncoder

one_hot = OneHotEncoder(sparse=False)
temp = one_hot.fit_transform(df.origin_le.values.reshape(-1,1))

In [30]:
df.origin.nunique()

9

In [29]:
temp.shape

(324, 9)

In [31]:
df_temp = pd.DataFrame(data=temp, columns=['origin=%d' % i for i in range(9)])

In [33]:
df_temp.head()

,origin=0,origin=1,origin=2,origin=3,origin=4,origin=5,origin=6,origin=7,origin=8
0,0,0,0,1,0,0,0,0,0
1,0,0,0,1,0,0,0,0,0
2,0,0,0,1,0,0,0,0,0
3,0,0,0,1,0,0,0,0,0
4,0,0,0,1,0,0,0,0,0


Как вы поняли, нужно было от строк перейти к числам и только потом использовать OneHot...

Можно сократить! Использовав DictVectorizer

### Более умная кодировка

#### Среднее по категории

Давайте закодируем категорию средним (например) значением какой-нибудь связяной с ней величины

In [37]:
df_offers.head()
# например, со значениями в признаке varietal мы можем связать среднее
# величину дисконта

,offer_id,campaign,varietal,min_qty,discount,origin,past_peak
0,1,January,Malbec,72,56,France,False
1,2,January,Pinot Noir,72,17,France,False
2,3,February,Espumante,144,32,Oregon,True
3,4,February,Champagne,72,48,France,True
4,5,February,Cabernet Sauvignon,144,44,New Zealand,True


In [38]:
mean_varietal_discount = df_offers.groupby('varietal')\
         .agg({'discount': np.mean})

In [41]:
mean_varietal_discount.columns = ['mean_var_disc']

In [42]:
mean_varietal_discount.head()

,mean_var_disc
varietal,
Cabernet Sauvignon,50.833333
Champagne,69.857143
Chardonnay,48.000000
Espumante,42.333333
Malbec,55.000000


In [62]:
df_extra = df.merge(mean_varietal_discount, left_on='varietal', right_index=True)

In [63]:
df_extra.head()

,customer_name,offer_id,campaign,varietal,min_qty,discount_x,origin,past_peak,varietal + origin,varietal_origin,origin_le,discount_y,mean_var_disc_x,mean_var_disc_y
0,Smith,2,January,Pinot Noir,72,17,France,False,Pinot Noir+France,Pinot Noir+France,3,45.25,45.25,45.25
1,Rodriguez,2,January,Pinot Noir,72,17,France,False,Pinot Noir+France,Pinot Noir+France,3,45.25,45.25,45.25
2,Martin,2,January,Pinot Noir,72,17,France,False,Pinot Noir+France,Pinot Noir+France,3,45.25,45.25,45.25
3,Jackson,2,January,Pinot Noir,72,17,France,False,Pinot Noir+France,Pinot Noir+France,3,45.25,45.25,45.25
4,Campbell,2,January,Pinot Noir,72,17,France,False,Pinot Noir+France,Pinot Noir+France,3,45.25,45.25,45.25


#### Вероятность категории

In [ ]:
# ?

#### TF-IDF для категориальных переменных

In [45]:
df_temp.head()
# term frequency and inversed document frequency
# tf-idf перевзвешивает эту матрицу
# количество слов в документе и количество документов, в которых встречается слово

,origin=0,origin=1,origin=2,origin=3,origin=4,origin=5,origin=6,origin=7,origin=8
0,0,0,0,1,0,0,0,0,0
1,0,0,0,1,0,0,0,0,0
2,0,0,0,1,0,0,0,0,0
3,0,0,0,1,0,0,0,0,0
4,0,0,0,1,0,0,0,0,0


In [46]:
from sklearn.feature_extraction.text import TfidfTransformer

In [47]:
tfidf = TfidfTransformer()

In [52]:
temp = tfidf.fit_transform(df_temp.values)

In [50]:
temp = np.asarray(temp)

In [51]:
temp

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  1., ...,  0.,  0.,  0.],
       [ 0.,  0.,  1., ...,  0.,  0.,  0.],
       [ 0.,  0.,  1., ...,  0.,  0.,  0.]])

### Разложение количественной переменной в "корзины"

In [53]:
df_offers.head()

,offer_id,campaign,varietal,min_qty,discount,origin,past_peak
0,1,January,Malbec,72,56,France,False
1,2,January,Pinot Noir,72,17,France,False
2,3,February,Espumante,144,32,Oregon,True
3,4,February,Champagne,72,48,France,True
4,5,February,Cabernet Sauvignon,144,44,New Zealand,True


In [54]:
df_offers.min_qty.describe()

count     32.000000
mean      65.812500
std       50.785722
min        6.000000
25%       12.000000
50%       72.000000
75%       72.000000
max      144.000000
Name: min_qty, dtype: float64

In [59]:
# аналог в пандасе
pd.cut(df_offers.min_qty, 4, labels=range(4))
# или pd.cut(df_offers.min_qty, [0, 50, 100, 150], right=False, include_lowest=True)
# pd.qcut(df_offers.min_qty, q=2) - по квантилями, а не по количеству
# pd.qcut(df_offers.min_qty, q=[0.05, 0.2, 0.8, 1]) - прямо задать квантили

0     1
1     1
2     3
3     1
4     3
5     3
6     0
7     0
8     3
9     1
10    1
11    1
12    0
13    1
14    3
15    1
16    0
17    0
18    0
19    1
20    0
21    1
22    3
23    0
24    1
25    3
26    1
27    0
28    0
29    0
30    1
31    1
Name: min_qty, dtype: category
Categories (4, int64): [0 < 1 < 2 < 3]

# Собственно, кластеризация

Теперь, используя весь этот багаж знаний, кластеризуйте-таки покупателей и постарайтесь проинтерпретировать кластеры

Можно использовать самый наивный метод - метод kMeans. Но лучше, если вы с ним уже знакомы, аггломеративную кластеризацию.

In [60]:
df_transactions.customer_name.nunique()

100

## Предлагается задать следующие признаки:
* средний размер дисконта для сорта + страны происхождения
* сорта купленных вин
* среднее значение min_qty

Должна получиться таблица 100*D

In [85]:
df_start[df_start.varietal == 'Espumante']

,customer_name,offer_id,campaign,varietal,min_qty,discount,origin,past_peak
44,Williams,18,July,Espumante,6,50,Oregon,False
45,Taylor,18,July,Espumante,6,50,Oregon,False
46,Walker,18,July,Espumante,6,50,Oregon,False
47,Perez,18,July,Espumante,6,50,Oregon,False
48,King,18,July,Espumante,6,50,Oregon,False
49,Adams,18,July,Espumante,6,50,Oregon,False
50,Hill,18,July,Espumante,6,50,Oregon,False
51,Rivera,18,July,Espumante,6,50,Oregon,False
52,James,18,July,Espumante,6,50,Oregon,False
53,Myers,18,July,Espumante,6,50,Oregon,False
